In [23]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import Literal

class FileType(Enum):
    PDF = 'pdf'
    CSV = 'csv'
    TXT = 'txt'
    MD = 'md'
    URL = 'url'
    PPTX = 'pptx'
    DOCX = 'docx'
    XLS = 'xls'
    XLSX = 'xlsx'
    XML = 'xml'
    GDOC = 'gdoc'
    GSHEET = 'gsheet'
    GSLIDE = 'gslide'
    GPDF = 'gpdf'
    YOUTUBE_URL = 'youtube_url'
    IMG = 'img'


In [11]:
class AIResistantArgs(BaseModel):
    topic: str = Field(..., min_length=1, max_length=255, description="Topic or subject related to the content")
    assignment: str = Field(..., min_length=1, max_length=255, description="The given assignment")
    grade_level: Literal["elementary", "middle", "high", "college", "professional"] = Field(..., description="Educational level to which the content is directed")
    file_type: str = Field(..., description="Type of file being handled, according to the defined enumeration")
    file_url: str = Field(..., description="URL or path of the file to be processed")
    lang: str = Field(..., description="Language in which the file or content is written")

In [13]:
input_data = AIResistantArgs(
    topic="Introduction to Data Science",
    assignment="Develop 4 algorithms for DS",
    grade_level="college",
    file_type="pdf",
    file_url="https://example.com/introduction_to_data_science.pdf",
    lang="en"
)

print(f"Topic: {input_data.topic}")
print(f"Grade Level: {input_data.grade_level}")
print(f"File Type: {input_data.file_type}")
print(f"File URL: {input_data.file_url}")
print(f"Language: {input_data.lang}")

Topic: Introduction to Data Science
Grade Level: college
File Type: pdf
File URL: https://example.com/introduction_to_data_science.pdf
Language: en


In [14]:
from typing import List

class AIResistanceIdea(BaseModel):
    assignment_description: str = Field(..., description="Detailed description of the modified assignment")
    explanation: str = Field(..., description="Explanation of how this modification makes the assignment AI-resistant")

class AIResistantOutput(BaseModel):
    topic: str = Field(..., description="Topic or subject related to the assignment")
    grade_level: str = Field(..., description="Educational level to which the assignment is directed")
    ideas: List[AIResistanceIdea] = Field(..., description="List of 3 ideas to make the assignment AI-resistant, including explanation")

In [19]:
class FileHandlerError(Exception):
    """Raised when a file content cannot be loaded. Used for tools which require file handling."""
    def __init__(self, message, url=None):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

class ImageHandlerError(Exception):
    """Raised when an image cannot be loaded. Used for tools which require image handling."""
    def __init__(self, message, url):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

class VideoTranscriptError(Exception):
    """Raised when a video transcript cannot be loaded. Used for tools which require video transcripts."""
    def __init__(self, message, url):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

In [24]:
%%writefile document_loaders.py
from langchain_community.document_loaders import YoutubeLoader, PyPDFLoader, TextLoader, UnstructuredURLLoader, UnstructuredPowerPointLoader, Docx2txtLoader, UnstructuredExcelLoader, UnstructuredXMLLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import os
import tempfile
import uuid
import requests
import gdown
from enum import Enum

class FileType(Enum):
    PDF = 'pdf'
    CSV = 'csv'
    TXT = 'txt'
    MD = 'md'
    URL = 'url'
    PPTX = 'pptx'
    DOCX = 'docx'
    XLS = 'xls'
    XLSX = 'xlsx'
    XML = 'xml'
    GDOC = 'gdoc'
    GSHEET = 'gsheet'
    GSLIDE = 'gslide'
    GPDF = 'gpdf'
    YOUTUBE_URL = 'youtube_url'
    IMG = 'img'

STRUCTURED_TABULAR_FILE_EXTENSIONS = {"csv", "xls", "xlsx", "gsheet", "xml"}

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

def read_text_file(file_path):
    # Get the directory containing the script file
    script_dir = os.path.dirname(os.path.abspath(__file__))

    # Combine the script directory with the relative file path
    absolute_file_path = os.path.join(script_dir, file_path)

    with open(absolute_file_path, 'r') as file:
        return file.read()

def get_docs(file_url: str, file_type: str, verbose=True):
    file_type = file_type.lower()
    try:
        file_loader = file_loader_map[FileType(file_type)]
        docs = file_loader(file_url, verbose)

        return docs

    except Exception as e:
        print(e)
        print(f"Unsupported file type: {file_type}")
        raise FileHandlerError(f"Unsupported file type", file_url) from e

class FileHandler:
    def __init__(self, file_loader, file_extension):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):
        # Generate a unique filename with a UUID prefix
        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        # Download the file from the URL and save it to a temporary file
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful

        with tempfile.NamedTemporaryFile(delete=False, prefix=unique_filename) as temp_file:
            temp_file.write(response.content)
            temp_file_path = temp_file.name

        # Use the file_loader to load the documents
        try:
            loader = self.file_loader(file_path=temp_file_path)
        except Exception as e:
            print(f"No such file found at {temp_file_path}")
            raise FileHandlerError(f"No file found", temp_file_path) from e

        try:
            documents = loader.load()
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available", temp_file_path) from e

        # Remove the temporary file
        os.remove(temp_file_path)

        return documents

def load_pdf_documents(pdf_url: str, verbose=False):
    pdf_loader = FileHandler(PyPDFLoader, "pdf")
    docs = pdf_loader.load(pdf_url)

    if docs:
        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found PDF file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_csv_documents(csv_url: str, verbose=False):
    csv_loader = FileHandler(CSVLoader, "csv")
    docs = csv_loader.load(csv_url)

    if docs:
        if verbose:
            print(f"Found CSV file")
            print(f"Splitting documents into {len(docs)} chunks")

        return docs

def load_txt_documents(notes_url: str, verbose=False):
    notes_loader = FileHandler(TextLoader, "txt")
    docs = notes_loader.load(notes_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found TXT file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_md_documents(notes_url: str, verbose=False):
    notes_loader = FileHandler(TextLoader, "md")
    docs = notes_loader.load(notes_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found MD file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_url_documents(url: str, verbose=False):
    url_loader = UnstructuredURLLoader(urls=[url])
    docs = url_loader.load()

    if docs:
        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found URL")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_pptx_documents(pptx_url: str, verbose=False):
    pptx_handler = FileHandler(UnstructuredPowerPointLoader, 'pptx')

    docs = pptx_handler.load(pptx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found PPTX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_docx_documents(docx_url: str, verbose=False):
    docx_handler = FileHandler(Docx2txtLoader, 'docx')
    docs = docx_handler.load(docx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found DOCX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xls_documents(xls_url: str, verbose=False):
    xls_handler = FileHandler(UnstructuredExcelLoader, 'xls')
    docs = xls_handler.load(xls_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XLS file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xlsx_documents(xlsx_url: str, verbose=False):
    xlsx_handler = FileHandler(UnstructuredExcelLoader, 'xlsx')
    docs = xlsx_handler.load(xlsx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XLSX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xml_documents(xml_url: str, verbose=False):
    xml_handler = FileHandler(UnstructuredXMLLoader, 'xml')
    docs = xml_handler.load(xml_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XML file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

class FileHandlerForGoogleDrive:
    def __init__(self, file_loader, file_extension='docx'):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):

        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        try:
            gdown.download(url=url, output=unique_filename, fuzzy=True)
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available") from e

        try:
            loader = self.file_loader(file_path=unique_filename)
        except Exception as e:
            print(f"No such file found at {unique_filename}")
            raise FileHandlerError(f"No file found", unique_filename) from e

        try:
            documents = loader.load()
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available") from e

        os.remove(unique_filename)

        return documents

def load_gdocs_documents(drive_folder_url: str, verbose=False):

    gdocs_loader = FileHandlerForGoogleDrive(Docx2txtLoader)

    docs = gdocs_loader.load(drive_folder_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Docs files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gsheets_documents(drive_folder_url: str, verbose=False):
    gsheets_loader = FileHandlerForGoogleDrive(UnstructuredExcelLoader, 'xlsx')
    docs = gsheets_loader.load(drive_folder_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Sheets files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gslides_documents(drive_folder_url: str, verbose=False):
    gslides_loader = FileHandlerForGoogleDrive(UnstructuredPowerPointLoader, 'pptx')
    docs = gslides_loader.load(drive_folder_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Slides files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gpdf_documents(drive_folder_url: str, verbose=False):

    gpdf_loader = FileHandlerForGoogleDrive(PyPDFLoader,'pdf')

    docs = gpdf_loader.load(drive_folder_url)
    if docs:

        if verbose:
            print(f"Found Google PDF files")
            print(f"Splitting documents into {len(docs)} chunks")

        return docs

def load_docs_youtube_url(youtube_url: str, verbose=True) -> str:
    try:
        loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=True)
    except Exception as e:
        print(f"No such video found at {youtube_url}")
        raise VideoTranscriptError(f"No video found", youtube_url) from e

    try:
        docs = loader.load()
        length = docs[0].metadata["length"]
        title = docs[0].metadata["title"]

    except Exception as e:
        print(f"Video transcript might be private or unavailable in 'en' or the URL is incorrect.")
        raise VideoTranscriptError(f"No video transcripts available", youtube_url) from e

    if verbose:
        print(f"Found video with title: {title} and length: {length}")
        print(f"Combined documents into a single string.")
        print(f"Beginning to process transcript...")

    split_docs = splitter.split_documents(docs)

    return split_docs

llm_for_img = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def generate_docs_from_img(img_url, verbose: bool=False):
    message = HumanMessage(
    content=[
            {
                "type": "text",
                "text": "Give me a summary of what you see in the image. It must be 3 detailed paragraphs.",
            },
            {"type": "image_url", "image_url": img_url},
        ]
    )

    try:
        response = llm_for_img.invoke([message]).content
        print(f"Generated summary: {response}")
        docs = Document(page_content=response, metadata={"source": img_url})
        split_docs = splitter.split_documents([docs])
    except Exception as e:
        print(f"Error processing the request due to Invalid Content or Invalid Image URL")
        raise ImageHandlerError(f"Error processing the request", img_url) from e

    return split_docs

file_loader_map = {
    FileType.PDF: load_pdf_documents,
    FileType.CSV: load_csv_documents,
    FileType.TXT: load_txt_documents,
    FileType.MD: load_md_documents,
    FileType.URL: load_url_documents,
    FileType.PPTX: load_pptx_documents,
    FileType.DOCX: load_docx_documents,
    FileType.XLS: load_xls_documents,
    FileType.XLSX: load_xlsx_documents,
    FileType.XML: load_xml_documents,
    FileType.GDOC: load_gdocs_documents,
    FileType.GSHEET: load_gsheets_documents,
    FileType.GSLIDE: load_gslides_documents,
    FileType.GPDF: load_gpdf_documents,
    FileType.YOUTUBE_URL: load_docs_youtube_url,
    FileType.IMG: generate_docs_from_img
}

Overwriting document_loaders.py


In [7]:
!pip install langchain langchain-core langchain-google-genai langchain_community langchain-chroma chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!pip install pypdf fpdf youtube-transcript-api pytube unstructured python-pptx docx2txt networkx pandas xlrd openpyxl gdown pytest PyPDF2 psutil

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.8 MB/s eta 0:00:0

In [9]:
!pip uninstall -y nltk
!pip install nltk
import nltk
nltk.download('punkt')

Found existing installation: nltk 3.8.1
Uninstalling nltk-3.8.1:
  Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
from typing import List, Dict
import os

from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [33]:
def read_text_file(file_path):
    # Get the current working directory
    script_dir = os.getcwd()

    # Combine the script directory with the relative file path
    absolute_file_path = os.path.join(script_dir, file_path)

    try:
        with open(absolute_file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        # Handle the case where the file is not found
        print(f"File not found: {absolute_file_path}")
        return None

In [39]:
class QuizBuilder:
    def __init__(self, args=None, vectorstore_class=Chroma, prompt=None, embedding_model=None, model=None, parser=None, verbose=False):
        default_config = {
            "model": GoogleGenerativeAI(model="gemini-1.5-flash"),
            "embedding_model": GoogleGenerativeAIEmbeddings(model='models/embedding-001'),
            "parser": JsonOutputParser(pydantic_object=AIResistantOutput),
            "prompt": read_text_file("prompt/ai-resistant-prompt.txt"),
            "vectorstore_class": Chroma
        }

        self.prompt = prompt or default_config["prompt"]
        self.model = model or default_config["model"]
        self.parser = parser or default_config["parser"]
        self.embedding_model = embedding_model or default_config["embedding_model"]

        self.vectorstore_class = vectorstore_class or default_config["vectorstore_class"]
        self.vectorstore, self.retriever, self.runner = None, None, None
        self.args = args
        self.verbose = verbose

        if vectorstore_class is None: raise ValueError("Vectorstore must be provided")
        if args.topic is None: raise ValueError("Topic must be provided")
        if args.assignment is None: raise ValueError("Assignment must be provided")
        if args.grade_level is None: raise ValueError("Grade Level must be provided")
        if args.grade_level is None: raise ValueError("Language must be provided")


    def compile(self, documents: List[Document]):
        # Return the chain
        prompt = PromptTemplate(
            template=self.prompt,
            input_variables=["attribute_collection"],
            partial_variables={"format_instructions": self.parser.get_format_instructions()}
        )

        if self.runner is None:
            print(f"Creating vectorstore from {len(documents)} documents") if self.verbose else None
            self.vectorstore = self.vectorstore_class.from_documents(documents, self.embedding_model)
            print(f"Vectorstore created") if self.verbose else None

            self.retriever = self.vectorstore.as_retriever()
            print(f"Retriever created successfully") if self.verbose else None

            self.runner = RunnableParallel(
                {"context": self.retriever,
                "attribute_collection": RunnablePassthrough()
                }
            )

        chain = self.runner | prompt | self.model | self.parser

        if self.verbose: print(f"Chain compilation complete")

        return chain

    def create_assignments(self, documents: List[Document]):
        if self.verbose: print(f"Creating the AI-Resistant assignments")

        chain = self.compile(documents)

        response = chain.invoke(f"Topic: {self.args.topic}, Assignment: {self.args.assignment}, Grade Level: {self.args.grade_level}, Lang: {self.args.lang}")

        if self.verbose: print(f"Deleting vectorstore")
        self.vectorstore.delete_collection()

        return response

In [26]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [27]:
from document_loaders import get_docs

input_data = AIResistantArgs(
    topic="Introduction to Data Science",
    assignment="Develop a Neural Network Architecture for emulating Data Science Workflow",
    grade_level="college",
    file_type="pdf",
    file_url="http://ijsmsjournal.org/2021/volume-4%20issue-4/ijsms-v4i4p137.pdf",
    lang="en"
)

docs = get_docs(input_data.file_url, input_data.file_type, True)

Found PDF file
Splitting documents into 13 chunks


In [35]:
docs

[Document(metadata={'source': '/tmp/1b3a99dd-d4e2-46ee-9d54-fe7d1bc035e3.pdfmaj72wsu', 'page': 0}, page_content='International Journal of Science and Management Studies (IJSMS)                        E-ISSN: 2581 -5946  \nDOI: 10.51386/25815946/ijsms -v4i4p13 7 \nVolume: 4 Issue: 4                                   July to August 2021                             https://www.ijsmsjournal.org  \n  \n \n                                       This is an open access article under the CC BY -NC-ND license (http://creativecommons.org/licenses/by -nc-nd/4.0/ )      Page 407  Introduction to Data Science - An Overview  \nThangakumar J eyaprakash#1, Padmaveni K#2   \n#1#2Associate Professor and Department of Computer Science and Engineering  \n#1#2Hindustan Institute of Technology and Science, India  \n \nAbstract Data science plays a vital role in the research field of computer science and engineering which \ninvolves collection of data, transformation, processing, describing,  and modelling. I

In [30]:
%%writefile ai-resistant-prompt.txt
You are an expert in generating educational assignments resistant to AI tools. Generate three different versions of the uploaded assignment,
each modified to be AI-resistant while maintaining the core educational objectives. Each version should include varied modifications that make it challenging
for AI tools to solve or generate responses. Provide a detailed explanation for each version, specifically highlighting how and why the changes make the assignment
resistant to AI tools, considering limitations such as understanding nuanced questions, producing original thought-provoking responses, or solving complex problems.

Here is the topic, assignment, grade level, and language of the assignment:
{attribute_collection}

Your response should be in the following format:
{format_instructions}

Writing ai-resistant-prompt.txt


In [40]:
output = QuizBuilder(args=input_data, verbose=True).create_assignments(docs)

Creating the AI-Resistant assignments
Creating vectorstore from 13 documents
Vectorstore created
Retriever created successfully
Chain compilation complete
Deleting vectorstore


In [41]:
output

{'topic': 'Introduction to Data Science',
 'grade_level': 'college',
 'ideas': [{'assignment_description': 'Develop a neural network architecture for emulating a specific Data Science workflow, but with the constraint that the network must be built using a specific, less commonly used framework or library. This framework should offer unique challenges compared to popular libraries like TensorFlow or PyTorch. For example, consider using a library like CNTK, Theano, or a specialized library focused on a specific type of neural network (e.g., Spiking Neural Networks). Analyze the strengths and weaknesses of this chosen framework in comparison to more popular options, discussing the trade-offs in design and implementation.',
   'explanation': 'This modification makes the assignment AI-resistant because it requires students to research and understand a less common framework, which AI tools are less likely to have been trained on. The assignment also asks for a comparative analysis, requirin